In [1]:
import numpy as np
from math import sqrt, atan
from random import random, randint, seed

pi = np.pi
twopi = 2 * np.pi

In [2]:
epsilon = 2**-32

In [3]:
import cv2 as cv
from matplotlib import pyplot as plt

class RasterImage():
    black_color = (   0,   0,   0 )
    white_color = ( 255, 255, 255 )
        
    def __init__(self, width, height, pen=None, bg=None, rgb=None):
        self.width = width
        self.height = height
        
        if pen is None:
            pen = self.black_color
        self.pen = pen
        
        if bg is None:
            bg = self.white_color
        self.bg = bg
        
        self.image = np.zeros((height, width, 3), np.uint8)

        if rgb is None:
            self.clear()
        else:
            self.paste(rgb)

    def paste(self, rgb):
        self.image[:, :] = rgb[:, :]
            
    def fill(self, rgb):
        self.image[:, :] = rgb
        
    def clear(self):
        self.fill(self.bg)        

    def swap_pen_bg(self):
        self.pen, self.bg = self.bg, self.pen
    
    def write(self, filename):
         cv.imwrite(filename, self.image)
        
    def show(self):
        plt.imshow(self.image)
    
    def set_bg(self, bg):
        self.bg = bg
    
    def set_pen(self, pen):
        self.pen = pen    

    def pick(self, x, y):
        return self.image[y, x]
        
    def plot(self, x, y, pen=None, erase=False):
        if pen is None:
            pen = self.pen
        if erase:
            pen = self.bg

        self.image[y, x] = pen

    def line(self, x1, y1, x2, y2, pen=None):

        # this is basically Bresenheim's algorithm without the integer arithmetic optimization
        
        def _sign(d):
            if d < 0:
                return -1
            elif d == 0:
                return 0
            else:
                return 1
    
        dx = x2 - x1
        dy = y2 - y1
        
        # always plot the starting point

        self.plot(x1, y1, pen)
        
        if dx != 0 or dy != 0:
            if abs(dx) > abs(dy):
                # plot "horizontal" line

                assert(dx != 0)
                slope = dy / abs(dx)
                dx = _sign(dx)
                x = x1 + dx
                y = float(y1) + slope

                while x != x2:
                    self.plot(x, int(y), pen)
                    x += dx
                    y += slope
            else:
                # plot "vertical" line
                
                assert(dy != 0)
                slope = dx / abs(dy)
                dy = _sign(dy)
                y = y1 + dy
                x = float(x1) + slope

                while y != y2:
                    self.plot(int(x), y, pen)
                    y += dy
                    x += slope
                
            # always plot end point, unless both dx,dy is 0
        
            self.plot(x2, y2, pen)
        else:
            pass # dx & dy = 0
    
    def box(self, x1, y1, x2, y2, pen=None):
        # draw box as horizontal lines
        
        dy = 1
        if y2 < y1:
            dy = -1
       
        y = y1
        while y != y2:
            self.line(x1, y, x2, y, pen)
            y += dy

        # always draw last line
            
        self.line(x1, y2, x2, y2, pen)

In [4]:
from scipy.signal import convolve2d

def gaussian_kernel(kernel_size, sigma, mu=0): 
    x, y = np.meshgrid(np.linspace(-1, 1, kernel_size),
                       np.linspace(-1, 1, kernel_size))
    
    tss = 2 * sigma ** 2
    d2 = (x - mu) ** 2 + (y - mu) ** 2
    amp = 1 / (twopi * tss)

    return amp * np.exp(-d2 / tss)

def stencil5():
    s5A = [ 0 ] * 5
    s5B = s5A[:]
    s5B[2] = 1
    s5C = [ 0, 1, -4, 1, 0 ]

    return np.array([ s5A, s5B, s5C, s5B, s5A ])

def lp_laplacian(sigma):
    a = convolve2d(gaussian_kernel(15, sigma=sigma), stencil5(), mode="same", boundary="fill")
    b = convolve2d(gaussian_kernel(15, sigma=sigma), stencil5(), mode="same", boundary="fill")
    
    return convolve2d(a, b, mode="same", boundary="fill")

In [5]:
width, height = 200, 200

seed(1122032234)

cf = np.zeros((height, width), np.float64)


In [ ]:
N = 2000

ri = RasterImage(2*width, 2*height)

lp_laplacian_kernel = lp_laplacian(.22)

diffusion = 0.01

a = 6
A = 1 / atan(a)
q = 1

for i in range(N):
    if i % 17 == 0:
        x, y = randint(0, width - 1), randint(0, height - 1)
        s = randint(1, int(0.333 * min(width, height)))
        
        x, y = randint(0, width - s - 1), randint(0, height - s - 1)
        
        c = randint(0,99) / 100
        
        if q:
            c = -sqrt(c)
        else:
            c = sqrt(c)
    
        q = 1 - q
    
        for Y in range(y, min(y + s + 1, height)):
            row = cf[Y]

            for X in range(x, min(x + s + 1, width)):
                row[X] = c + 0.35 * row[X]

    for y in range(height):
        src_row = cf[y]
        
        for x in range(width):
            r, g, b = 0, 0, 0
            s = src_row[x]
            
            s = A * atan(a * s)
            
            if s < 0:
                b = int(-255 * s)
                if b > 255:
                    b = 255
            else:
                r = int(255 * s)
                if r > 255:
                    r = 255
            ri.plot(2*x,   2*y,   pen=(r, g, b))
            ri.plot(2*x+1, 2*y,   pen=(r, g, b))
            ri.plot(2*x,   2*y+1, pen=(r, g, b))
            ri.plot(2*x+1, 2*y+1, pen=(r, g, b))
            #if s != 0:
            #    print(s, x, y, r, g, b)
            #    slutaspela
        
            
    ri.write("cf/a{:0>5d}.png".format(i))
    
    laplacian = convolve2d(cf, lp_laplacian_kernel, mode="same", boundary="wrap")
    cf -= diffusion * laplacian
    
    cf *= 0.965
    
    np.clip(cf, -0.99, 0.99)